# Prototyping model2regex

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [2]:
import torch
from torch import nn, optim
from torch.distributions import Categorical
from random import Random
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from itertools import chain
import torch.nn.functional as F
import string

We going to try to learn a simple DGA called bajori

First we are generating a dataset of of domains

In [3]:
from dga import banjori, generate_dataset
domains: list[str] = generate_dataset(banjori,'earnestnessbiophysicalohax.com', 2**16)

We are going to generate a pytorch [Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) to help us doing the batching and data shuffling 

In [4]:
class Domains(Dataset):
    def __init__(self, data: list[str]):
        self.data = data
        self.chars = sorted(list(set(chain(*self.data))))
        self.vocabSize = len(self.chars) + 1
        self.max_size = len(max(self.data, key=lambda d: len(d)))
        self.char2ind = {ch : i for i,ch in enumerate(self.chars, start=1)}
        self.ind2char = {i : ch for i,ch in enumerate(self.chars, start=1)}

    def __len__(self):
        return len(self.data)

    def isEndChar(self, ind):
        return ind == 0
    def charTensor(self, _input):
        return torch.tensor([[self.char2ind[c] for c in _input]]).permute(1,0)
        
    def __getitem__(self, idx: int):
        item = torch.tensor([self.char2ind[c] for c in self.data[idx]])
        # we need tensors of same size, so if any domain has a different size we then pad it with 0 which will be our "end char"
        item = F.pad(item, (0,self.max_size - len(item)), "constant", 0)
        return (item, F.pad(item[1:], (0,1), "constant", 0))
        

In [5]:
dataset = Domains(domains)
x,y = next(iter(DataLoader(dataset, batch_size=64, shuffle=True)))
print("x:\n\t", x.shape)
print("y:\n\t", y.shape)
print(dataset.data[0])

x:
	 torch.Size([64, 30])
y:
	 torch.Size([64, 30])
kwtoestnessbiophysicalohax.com


Following the dataset we define a Model for learning the structure of our DGA

In [6]:
from torch import nn
class Model(nn.Module):
    def __init__(self, vocabSize, emb, size, nlayers):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocabSize,
                                      embedding_dim=emb)
        self.rnn = nn.GRU(input_size=emb, hidden_size=size,
                          num_layers=nlayers)
        self.decoder = nn.Linear(size, vocabSize)
        
    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.rnn(embedding, hidden_state)
        output = self.decoder(output)
        return output, hidden_state.detach()

In [7]:
# Trying to understand how the embedding put out the permuted input
dataset = Domains(domains)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
(x,y) = next(iter(dataloader))
print(x.permute(1,0))
embedding = nn.Embedding(num_embeddings=dataset.vocabSize, embedding_dim=64)
emb = embedding(x.permute(1,0))
emb.shape


tensor([[12,  8, 10,  ..., 27, 19, 23],
        [ 6, 22, 20,  ..., 25,  7, 25],
        [19,  5, 17,  ..., 23, 20,  6],
        ...,
        [ 4,  4,  4,  ...,  4,  4,  4],
        [16, 16, 16,  ..., 16, 16, 16],
        [14, 14, 14,  ..., 14, 14, 14]])


torch.Size([30, 64, 64])

In [8]:
class Trainer:
    def __init__(self, dataset: Domains, **kwargs):
        self.dataset = dataset
        self.hidden_size = kwargs.get('hidden_size', 128)
        self.num_layers = kwargs.get('num_layers', 1)
        self.embed_dim = kwargs.get('embed_dim', 64)
        self.device = kwargs.get('device', 'cuda:0')
        self.model = Model(dataset.vocabSize, self.embed_dim, self.hidden_size, self.num_layers)
        
    def predict(self, starter: str):
        chart = self.dataset.charTensor(starter)
        state = None
        device = self.device
        for _ in range(100):
            if state is not None:
                state = state.to(device)
            output, state = self.model(chart.to(device), state)
            output = F.softmax(torch.squeeze(output[-1, :]), dim=0)
            dist = Categorical(output)
            index = dist.sample()
            ind = index.item()
            if dataset.isEndChar(ind):
                break
            starter += self.dataset.ind2char[ind]
            chart = torch.tensor([[ind]])
        return starter 

    def train(self):
        criterion = nn.CrossEntropyLoss(reduction="mean")
        dataset = self.dataset
        device = self.device
        dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
        model = self.model
        model.to(device)
        model.train()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        h0 = None
        for epoch in range(100):
            for batch, (x,y) in enumerate(dataloader):
                optimizer.zero_grad()
                if h0 is not None:
                    h0 = h0.to(device)
                output, h0 = model(x.permute(1,0).to(device), h0)
                loss = criterion(output.permute(1,2,0), y.to(device))
                loss.backward()
                optimizer.step()
            print()
            print("----------------------------------------------")
            print({'epoch': epoch, 'batch': batch, 'loss': loss.item()})
            for char in string.ascii_lowercase:
                print(self.predict(char))
            print("----------------------------------------------")
            model.train()


In [9]:
HIDDEN_SIZE = 128
NLAYERS = 1
EMBEDDING_DIM = 64
trainer = Trainer(dataset, hidden_size=HIDDEN_SIZE, num_layers=NLAYERS, embed_dim=EMBEDDING_DIM)
trainer.train()


----------------------------------------------
{'epoch': 0, 'batch': 4095, 'loss': 0.3053528070449829}
asiophysicalohax.com
bicosscom
calohax.com
dz.com
estnessbiophysicalohax.com
fhnestnessbiophysicalohax.com
gestnessbiophysicalohax.com
hvx.com
icalohax.com
jtnestnessbiophysicalohax.com
kom
lm
mndvestnessbiophysicalohax.com
nbestnessbiophysicalohax.com
ornestnessbiophysicalohax.com
physicalohax.com
qlohax.com
rphysicalohax.com
suoiestnessbiophysicalohax.com
tnestnessbiophysicalohax.com
usgestnessbiophysicalohax.com
vphysicalohax.com
w
x.com
ysicalohax.com
z.com
----------------------------------------------


KeyboardInterrupt: 

In [ ]:
class DGAClassifier(nn.Module):
    def __init__(self, vocabSize, emb, size, nlayers):
        super(DGAClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocabSize,
                                      embedding_dim=emb)
        self.rnn = nn.GRU(input_size=emb, hidden_size=size,
                          num_layers=nlayers)
        self.linear = nn.Linear(in_features=size, out_features=1)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.rnn(embedding, hidden_state)
        soft = self.linear(hidden_state[-1])
        return soft, hidden_state.detach()

model = DGAClassifier(dataset.vocabSize, EMBEDDING_DIM, HIDDEN_SIZE, NLAYERS)
out = model(dataset.charTensor(domains[0]), None)


In [ ]:
import pandas as pd
from pathlib import Path
top1m = pd.read_csv(Path('data/top-1m.csv'))
real_domains = top1m.values[:,1]

In [ ]:
dga_dataset = list(zip(domains, [1 for _ in range(len(domains))]))
dga_dataset.extend(list(zip(real_domains, [0 for _ in range(len(real_domains))])))
import random
random.choices(k=20, population=dga_dataset)

In [ ]:
class DomainsAndDGA(Dataset):
    def __init__(self, dga_domains: list[str], real_domains_file='data/top-1m.csv'):
        top1m = pd.read_csv(real_domains_file)
        real_domains = top1m.values[:,1]
        self.max_size = len(max(chain(real_domains, dga_domains), key=lambda d: len(d)))
        self.data = list(zip(real_domains, [1 for _ in range(len(real_domains))]))
        self.data.extend(list(zip(dga_domains, [0 for _ in range(len(dga_domains))])))
        self.chars = sorted(list(set(chain(*[d[0] for d in self.data]))))
        self.vocabSize = len(self.chars) + 1
        self.char2ind = {ch : i for i,ch in enumerate(self.chars, start=1)}
        self.ind2char = {i : ch for i,ch in enumerate(self.chars, start=1)}

    def __len__(self):
        return len(self.data)

    def isEndChar(self, ind):
        return ind == 0
    def charTensor(self, _input):
        return torch.tensor([[self.char2ind[c] for c in _input]]).permute(1,0)
        
    def __getitem__(self, idx: int):
        item, label = self.data[idx]
        item = torch.tensor([self.char2ind[c] for c in item])
        # we need tensors of same size, so if any domain has a different size we then pad it with 0 which will be our "end char"
        item = F.pad(item, (0,self.max_size - len(item)), "constant", 0)
        return (item, label)

In [ ]:
dataset = DomainsAndDGA(dga_domains=domains)

In [ ]:
class Trainer:
    def __init__(self, dataset: Domains, **kwargs):
        self.dataset = dataset
        self.hidden_size = kwargs.get('hidden_size', 128)
        self.num_layers = kwargs.get('num_layers', 1)
        self.embed_dim = kwargs.get('embed_dim', 64)
        self.device = kwargs.get('device', 'cuda:0')
        self.model = DGAClassifier(dataset.vocabSize, self.embed_dim, self.hidden_size, self.num_layers)
        
    def predict(self, starter: str):
        chart = self.dataset.charTensor(starter)
        state = None
        device = self.device
        for _ in range(100):
            if state is not None:
                state = state.to(device)
            output, state = self.model(chart.to(device), state)
            output = F.softmax(torch.squeeze(output[-1, :]), dim=0)
            dist = Categorical(output)
            index = dist.sample()
            ind = index.item()
            if dataset.isEndChar(ind):
                break
            starter += self.dataset.ind2char[ind]
            chart = torch.tensor([[ind]])
        return starter 

    def train(self):
        criterion = nn.CrossEntropyLoss()
        dataset = self.dataset
        device = self.device
        dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
        model = self.model
        model.to(device)
        model.train()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        h0 = None
        for epoch in range(100):
            for batch, (x,y) in enumerate(dataloader):
                optimizer.zero_grad()
                if h0 is not None:
                    h0 = h0.to(device)
                output, h0 = model(x.permute(1,0).to(device), h0)
                loss = criterion(output, y.to(device))
                loss.backward()
                optimizer.step()
            model.train()

In [ ]:
trainer = Trainer(dataset)
trainer.train()